In [1]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine
from ansys.mapdl.core.errors import MapdlRuntimeError
from ansys.mapdl.core import launch_mapdl
import re
import os
import pandas as pd
pd.set_option('display.max_colwidth', 150)

In [2]:
#Variáveis
source_dir = 'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024'
material_yielding_stress = 355
p_u = 3479
MAPDL_RUN_LOCATION='D:/constructal_automate_analysis_files'
MAPDL_START_TIMEOUT=120

In [11]:
mapdl = launch_mapdl(
    port=50052,
    run_location=MAPDL_RUN_LOCATION,
    nproc=4,
    override=True,
    loglevel="INFO",
    start_timeout=MAPDL_START_TIMEOUT,
    cleanup_on_exit=True
)

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - /SHOW SWITCH PLOTS TO  PNG         - RASTER MODE.
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files


In [12]:
def calc_post_processing_frequency_values_arr_position_for_ultimate_load(mapdl, p_u, flag):
    first_to_last_arr_position = len(mapdl.post_processing.frequency_values)-2-flag
    first_to_last_load = mapdl.post_processing.frequency_values[first_to_last_arr_position]

    second_to_last_arr_position = len(mapdl.post_processing.frequency_values)-3-flag
    second_to_last_load = mapdl.post_processing.frequency_values[second_to_last_arr_position]

    diff_between_loads = first_to_last_load - second_to_last_load
    load_increment = p_u/400

    if diff_between_loads > load_increment:
        return first_to_last_arr_position
    else:
        return second_to_last_arr_position

In [38]:
def interagir_com_mapdl(mapdl, analysis_dir, analysis_file_name, analysis_db_path, flag):

    # Abrir a pasta correta
    mapdl.cwd(analysis_dir)
    mapdl.filname(fname=analysis_file_name, key=0)
    mapdl.resume(fname=analysis_file_name, ext = 'db')

    # Calcular Carga e Tensão
    position_n_u = calc_post_processing_frequency_values_arr_position_for_ultimate_load(mapdl, p_u, flag)
    n_u = mapdl.post_processing.frequency_values[position_n_u]
    sigma_u = n_u/float(9.8)

    #Calcular Deslocamento Máximo
    position_w_max = position_n_u + 1
    mapdl.post1()
    mapdl.set(lstep=1, sbstep=position_w_max)
    negative_z_deflection = min(mapdl.post_processing.nodal_displacement("Z"))
    abs_negative_z_deflection = abs(negative_z_deflection)

    positive_z_deflection = max(mapdl.post_processing.nodal_displacement("Z"))

    if abs_negative_z_deflection > positive_z_deflection:
        z_deflection = abs_negative_z_deflection
    else:
        z_deflection = positive_z_deflection
    mapdl.finish()

    # Gerar imagens
    VON_MISES_IMG_SUFFIX = '_von_mises_dist.png'
    W_IMG_SUFFIX = '_w_dist.png'

    von_mises_dist_img_path = analysis_db_path.replace('.db', VON_MISES_IMG_SUFFIX)
    w_dist_img_path = analysis_db_path.replace('.db', W_IMG_SUFFIX)

    mapdl.result.plot_principal_nodal_stress(
        position_n_u,
        "SEQV",
        lighting=False,
        cpos="iso",
        background="white",
        text_color="black",
        add_text=False,
        show_edges=True,          
        edge_color="black",       
        cmap="jet",              
        rng=[0, material_yielding_stress],
        line_width=1.0,           
        off_screen=True,
        # screenshot=von_mises_dist_img_path,
        show_scalar_bar=True,
        # show_displacement=True,
        # displacement_factor=2,
        scalar_bar_args={
            "title": "Tensões (MPa)",
            "vertical": False,
            "fmt":"%8.2f",
            "width": 0.4,
            "position_x": 0.55,
            "position_y": 0.15,
            "title_font_size": 20,
            "label_font_size": 16,
        }
    )

    mapdl.result.plot_principal_nodal_stress(
        position_n_u,
        "SEQV",
        lighting=False,
        cpos="iso",
        background="white",
        text_color="black",
        add_text=False,
        show_edges=True,          
        edge_color="black",       
        cmap="jet",              
        rng=[0, material_yielding_stress],
        line_width=1.0,           
        off_screen=True,
        screenshot=von_mises_dist_img_path,
        show_scalar_bar=True,
        # show_displacement=True,
        # displacement_factor=2,
        scalar_bar_args={
            "title": "Tensões (MPa)",
            "vertical": False,
            "fmt":"%8.2f",
            "width": 0.4,
            "position_x": 0.55,
            "position_y": 0.15,
            "title_font_size": 20,
            "label_font_size": 16,
        }
    )

    mapdl.result.plot_nodal_displacement(
        position_n_u,
        "UZ",
        lighting=False,
        cpos="iso",
        background="white",
        text_color="black",
        add_text=False,
        show_edges=True,
        edge_color="black",
        cmap="jet",
        line_width=1.0,
        off_screen=True,
        screenshot=w_dist_img_path,
        show_scalar_bar=True,
        scalar_bar_args={
            "title": "Deslocamento em Z (mm)",
            "vertical": False,
            "fmt":"%8.2f",
            "width": 0.4,
            "position_x": 0.55,
            "position_y": 0.15,
            "title_font_size": 20,
            "label_font_size": 16,
        }
    )

    return n_u, sigma_u, z_deflection, von_mises_dist_img_path, w_dist_img_path

In [35]:
def interagir_com_bd(n_u, sigma_u, w_max, w_dist_img_path):
    # Configuração de conexão com o banco de dados
    connection = pymysql.connect(
        host="localhost",        
        user="constructal_automate",     
        password="constructal_design_2024",    
        database="constructal_automate_results"
    )
    
    try:
        with connection.cursor() as cursor:
            sql = """
            UPDATE constructal_automate_results.cbeb_elastoplasticbuckling as epb
            SET epb.n_u = %s, epb.sigma_u = %s, epb.w_max = %s
            WHERE epb.w_dist_img_path = %s
            """
            valores = (n_u, sigma_u, w_max, w_dist_img_path)
    
            # Executa o comando
            cursor.execute(sql, valores)
    
            # Confirma a transação
            connection.commit()
    
            print(f"{cursor.rowcount} linhas foram atualizadas.")
    finally:
        # Fecha a conexão
        connection.close()

In [43]:
def executar_correcao(mapdl, analysis_dir, analysis_file_name, analysis_db_path, flag):
    n_u, sigma_u, w_max, von_mises_dist_img_path, w_dist_img_path = interagir_com_mapdl(mapdl, analysis_dir, analysis_file_name, analysis_db_path, flag)
    print(f'n_u = {n_u}')
    print(f'sigma_u = {sigma_u}')
    print(f'w_max = {w_max}')
    print(f'von_mises_dist_img_path = {von_mises_dist_img_path}')
    print(f'w_dist_img_path = {w_dist_img_path}')
    prosseguir = input('Prosseguir com a persistência no BD?')
    if prosseguir == 'S':
        interagir_com_bd(n_u, sigma_u, w_max, w_dist_img_path)

In [26]:
def recuperar_dados_de_analise_do_bd():
    connection = pymysql.connect(
        host="localhost",        
        user="constructal_automate",     
        password="constructal_design_2024",    
        database="constructal_automate_results"
    )
    try:
        with connection.cursor() as cursor:
            # Query SQL
            sql = """
            SELECT spa.analysis_dir_path, REGEXP_SUBSTR(epb.w_dist_img_path, '[^/]+(?=_w_dist)') as analysis_file_name, REPLACE(spa.analysis_lgw_file_path, 'txt','db') as analysis_db_path FROM constructal_automate_results.cbeb_elastoplasticbuckling as epb
            INNER JOIN constructal_automate_results.cbeb_stiffenedplateanalysis as spa on epb.stiffened_plate_analysis_id=spa.id
            INNER JOIN constructal_automate_results.csg_stiffenedplate as sp on spa.stiffened_plate_id=sp.id
            WHERE
            spa.buckling_load_type_id = 2 and
            epb.id not between 259 and 266 and
            sp.id in (14, 22, 37, 46, 69, 79, 94, 101, 105, 107,113, 121, 128, 129);
            """
    
            # Executa a consulta
            cursor.execute(sql)
    
            # Recupera os resultados
            resultados = cursor.fetchall()
            analysis_dir_path_arr = [row[0] for row in resultados]
            analysis_file_name_arr = [row[1] for row in resultados]
            analysis_db_path_arr = [row[2] for row in resultados]
    finally:
        # Fecha a conexão
        connection.close()
    return analysis_dir_path_arr, analysis_file_name_arr, analysis_db_path_arr

In [27]:
analysis_dir_path_arr, analysis_file_name_arr, analysis_db_path_arr = recuperar_dados_de_analise_do_bd()

In [28]:
analysis_dir_path_arr

['D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L2T3_k1.72_MS25_SP14',
 'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L2T4_k1.4_MS25_SP22',
 'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L3T2_k3.1_MS25_SP37',
 'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L3T3_k1.56_MS25_SP46',
 'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L4T2_k2.7_MS25_SP69',
 'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L4T3_k1_MS25_SP79',
 'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L4T5_k1_MS25_SP94',
 'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L5T2_k2.4_MS25_SP101',
 'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L5T2_k0.625_MS25_SP105',
 'D:/constructal_automate_analysis_files/ec_2a_2_2_ph

In [29]:
analysis_file_name_arr

['BL2M1P0.3L2T3_k1.72_MS25_SP14',
 'BL2M1P0.3L2T4_k1.4_MS25_SP22',
 'BL2M1P0.3L3T2_k3.1_MS25_SP37',
 'BL2M1P0.3L3T3_k1.56_MS25_SP46',
 'BL2M1P0.3L4T2_k2.7_MS25_SP69',
 'BL2M1P0.3L4T3_k1_MS25_SP79',
 'BL2M1P0.3L4T5_k1_MS25_SP94',
 'BL2M1P0.3L5T2_k2.4_MS25_SP101',
 'BL2M1P0.3L5T2_k0.625_MS25_SP105',
 'BL2M1P0.3L5T3_k7.8_MS25_SP107',
 'BL2M1P0.3L5T3_k0.525_MS25_SP113',
 'BL2M1P0.3L5T4_k0.45_MS25_SP121',
 'BL2M1P0.3L5T5_k0.485_MS25_SP128',
 'BL2M1P0.3L5T5_k0.375_MS25_SP129']

In [30]:
analysis_db_path_arr

['D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L2T3_k1.72_MS25_SP14/BL2M1P0.3L2T3_k1.72_MS25_SP14.db',
 'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L2T4_k1.4_MS25_SP22/BL2M1P0.3L2T4_k1.4_MS25_SP22.db',
 'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L3T2_k3.1_MS25_SP37/BL2M1P0.3L3T2_k3.1_MS25_SP37.db',
 'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L3T3_k1.56_MS25_SP46/BL2M1P0.3L3T3_k1.56_MS25_SP46.db',
 'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L4T2_k2.7_MS25_SP69/BL2M1P0.3L4T2_k2.7_MS25_SP69.db',
 'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L4T3_k1_MS25_SP79/BL2M1P0.3L4T3_k1_MS25_SP79.db',
 'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L4T5_k1_MS25_SP94/BL2M1P0.3L4T5_k1_MS25_SP94.db',
 'D:/constructal_automate_analysis_files/ec_2

In [44]:
for analysis_dir_path, analysis_file_name, analysis_db_path in zip(analysis_dir_path_arr, analysis_file_name_arr, analysis_db_path_arr):
    executar_correcao(mapdl, analysis_dir_path, analysis_file_name, analysis_db_path, 1)

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Current working directory switched to D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L2T3_k1.72_MS25_SP14
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - CURRENT JOBNAME REDEFINED AS BL2M1P0.3L2T3_k1.72_MS25_SP14
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - RESUME ANSYS DATA FROM FILE NAME=BL2M1P0.3L2T3_k1.72_MS25_SP14.db                                                                                                                                                                                                                                    

 *** MAPDL GLOBAL STATUS *** 

 TITLE = BL2M1P0.3L2T3_k1.72_MS25_SP14                                         
 ANALYSIS TYPE =  STATIC (STEADY-STATE)
        36 KEYPOINTS DEFINED     MAX KEYPOINT NUMBER =             36
        64 LINES DEFINED         MAX LINE NUMBER =                 78
        29 AREAS DEFINED         MAX AREA NUMBER =               

Widget(value='<iframe src="http://localhost:63992/index.html?ui=P_0x1c1c2306de0_6&reconnect=auto" class="pyvis…

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L2T3_k1.72_MS25_SP14
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_2_2_phi_0.3_lancanova_2024\BL2M1P0.3L2T3_k1.72_MS25_SP14


Widget(value='<iframe src="http://localhost:63992/index.html?ui=P_0x1c1b5b9adb0_7&reconnect=auto" class="pyvis…

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L2T3_k1.72_MS25_SP14
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_2_2_phi_0.3_lancanova_2024\BL2M1P0.3L2T3_k1.72_MS25_SP14


...

Widget(value='<iframe src="http://localhost:63992/index.html?ui=P_0x1c299652e10_46&reconnect=auto" class="pyvi…

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L5T5_k0.375_MS25_SP129
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_2_2_phi_0.3_lancanova_2024\BL2M1P0.3L5T5_k0.375_MS25_SP129


Widget(value='<iframe src="http://localhost:63992/index.html?ui=P_0x1c2996526c0_47&reconnect=auto" class="pyvi…

n_u = 758.422
sigma_u = 77.39
w_max = 33.80053266554987
von_mises_dist_img_path = D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L5T5_k0.375_MS25_SP129/BL2M1P0.3L5T5_k0.375_MS25_SP129_von_mises_dist.png
w_dist_img_path = D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L5T5_k0.375_MS25_SP129/BL2M1P0.3L5T5_k0.375_MS25_SP129_w_dist.png


Prosseguir com a persistência no BD? n


In [46]:
executar_correcao(mapdl, analysis_dir_path_arr[11], analysis_file_name_arr[11], analysis_db_path_arr[11], 1)

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Current working directory switched to D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L5T5_k0.485_MS25_SP128
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - CURRENT JOBNAME REDEFINED AS BL2M1P0.3L5T4_k0.45_MS25_SP121
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - RESUME ANSYS DATA FROM FILE NAME=BL2M1P0.3L5T4_k0.45_MS25_SP121.db                                                                                                                                                                                                                                   

 *** MAPDL GLOBAL STATUS *** 

 TITLE = BL2M1P0.3L5T4_k0.45_MS25_SP121                                        
 ANALYSIS TYPE =  STATIC (STEADY-STATE)
        80 KEYPOINTS DEFINED     MAX KEYPOINT NUMBER =             80
       158 LINES DEFINED         MAX LINE NUMBER =                180
        79 AREAS DEFINED         MAX AREA NUMBER =            

Widget(value='<iframe src="http://localhost:63992/index.html?ui=P_0x1c2bc7159d0_48&reconnect=auto" class="pyvi…

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L5T4_k0.45_MS25_SP121
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_2_2_phi_0.3_lancanova_2024\BL2M1P0.3L5T4_k0.45_MS25_SP121


Widget(value='<iframe src="http://localhost:63992/index.html?ui=P_0x1c180940830_49&reconnect=auto" class="pyvi…

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L5T4_k0.45_MS25_SP121
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_2_2_phi_0.3_lancanova_2024\BL2M1P0.3L5T4_k0.45_MS25_SP121


Widget(value='<iframe src="http://localhost:63992/index.html?ui=P_0x1c2bc6c8dd0_50&reconnect=auto" class="pyvi…

n_u = 849.04995
sigma_u = 86.63775
w_max = 37.58244614082884
von_mises_dist_img_path = D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L5T4_k0.45_MS25_SP121/BL2M1P0.3L5T4_k0.45_MS25_SP121_von_mises_dist.png
w_dist_img_path = D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L5T4_k0.45_MS25_SP121/BL2M1P0.3L5T4_k0.45_MS25_SP121_w_dist.png


Prosseguir com a persistência no BD? S


1 linhas foram atualizadas.


In [47]:
def recuperar_dados_de_analise_do_bd_flag_1():
    connection = pymysql.connect(
        host="localhost",        
        user="constructal_automate",     
        password="constructal_design_2024",    
        database="constructal_automate_results"
    )
    try:
        with connection.cursor() as cursor:
            # Query SQL
            sql = """
            SELECT spa.analysis_dir_path, REGEXP_SUBSTR(epb.w_dist_img_path, '[^/]+(?=_w_dist)') as analysis_file_name, REPLACE(spa.analysis_lgw_file_path, 'txt','db') as analysis_db_path FROM constructal_automate_results.cbeb_elastoplasticbuckling as epb
            INNER JOIN constructal_automate_results.cbeb_stiffenedplateanalysis as spa on epb.stiffened_plate_analysis_id=spa.id
            INNER JOIN constructal_automate_results.csg_stiffenedplate as sp on spa.stiffened_plate_id=sp.id
            WHERE
            spa.buckling_load_type_id = 2 and
            epb.id not between 259 and 266 and
            sp.id in (22, 79, 94, 101, 105, 113, 129);
            """
    
            # Executa a consulta
            cursor.execute(sql)
    
            # Recupera os resultados
            resultados = cursor.fetchall()
            analysis_dir_path_arr = [row[0] for row in resultados]
            analysis_file_name_arr = [row[1] for row in resultados]
            analysis_db_path_arr = [row[2] for row in resultados]
    finally:
        # Fecha a conexão
        connection.close()
    return analysis_dir_path_arr, analysis_file_name_arr, analysis_db_path_arr

In [49]:
analysis_dir_path_arr, analysis_file_name_arr, analysis_db_path_arr = recuperar_dados_de_analise_do_bd_flag_1()

In [50]:
analysis_dir_path_arr

['D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L2T4_k1.4_MS25_SP22',
 'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L4T3_k1_MS25_SP79',
 'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L4T5_k1_MS25_SP94',
 'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L5T2_k2.4_MS25_SP101',
 'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L5T2_k0.625_MS25_SP105',
 'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L5T3_k0.525_MS25_SP113',
 'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L5T5_k0.375_MS25_SP129']

In [51]:
for analysis_dir_path, analysis_file_name, analysis_db_path in zip(analysis_dir_path_arr, analysis_file_name_arr, analysis_db_path_arr):
    executar_correcao(mapdl, analysis_dir_path, analysis_file_name, analysis_db_path, 0)

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Current working directory switched to D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L3T2_k3.1_MS25_SP37
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - CURRENT JOBNAME REDEFINED AS BL2M1P0.3L2T4_k1.4_MS25_SP22
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - RESUME ANSYS DATA FROM FILE NAME=BL2M1P0.3L2T4_k1.4_MS25_SP22.db                                                                                                                                                                                                                                     

 *** MAPDL GLOBAL STATUS *** 

 TITLE = BL2M1P0.3L2T4_k1.4_MS25_SP22                                          
 ANALYSIS TYPE =  STATIC (STEADY-STATE)
        44 KEYPOINTS DEFINED     MAX KEYPOINT NUMBER =             44
        80 LINES DEFINED         MAX LINE NUMBER =                 96
        37 AREAS DEFINED         MAX AREA NUMBER =                 

Widget(value='<iframe src="http://localhost:63992/index.html?ui=P_0x1c2bc6ca960_51&reconnect=auto" class="pyvi…

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L2T4_k1.4_MS25_SP22
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_2_2_phi_0.3_lancanova_2024\BL2M1P0.3L2T4_k1.4_MS25_SP22


Widget(value='<iframe src="http://localhost:63992/index.html?ui=P_0x1c180942600_52&reconnect=auto" class="pyvi…

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L2T4_k1.4_MS25_SP22
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_2_2_phi_0.3_lancanova_2024\BL2M1P0.3L2T4_k1.4_MS25_SP22


...

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L5T5_k0.375_MS25_SP129
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_2_2_phi_0.3_lancanova_2024\BL2M1P0.3L5T5_k0.375_MS25_SP129


Widget(value='<iframe src="http://localhost:63992/index.html?ui=P_0x1c2a8af5760_71&reconnect=auto" class="pyvi…

n_u = 787.77606
sigma_u = 80.38531224489796
w_max = 40.94198738578119
von_mises_dist_img_path = D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L5T5_k0.375_MS25_SP129/BL2M1P0.3L5T5_k0.375_MS25_SP129_von_mises_dist.png
w_dist_img_path = D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L5T5_k0.375_MS25_SP129/BL2M1P0.3L5T5_k0.375_MS25_SP129_w_dist.png


Prosseguir com a persistência no BD? S


0 linhas foram atualizadas.


In [53]:
def recuperar_dados_de_analise_do_bd_flag_2():
    connection = pymysql.connect(
        host="localhost",        
        user="constructal_automate",     
        password="constructal_design_2024",    
        database="constructal_automate_results"
    )
    try:
        with connection.cursor() as cursor:
            # Query SQL
            sql = """
            SELECT spa.analysis_dir_path, REGEXP_SUBSTR(epb.w_dist_img_path, '[^/]+(?=_w_dist)') as analysis_file_name, REPLACE(spa.analysis_lgw_file_path, 'txt','db') as analysis_db_path FROM constructal_automate_results.cbeb_elastoplasticbuckling as epb
            INNER JOIN constructal_automate_results.cbeb_stiffenedplateanalysis as spa on epb.stiffened_plate_analysis_id=spa.id
            INNER JOIN constructal_automate_results.csg_stiffenedplate as sp on spa.stiffened_plate_id=sp.id
            WHERE
            spa.buckling_load_type_id = 2 and
            epb.id not between 259 and 266 and
            sp.id = 107;
            """
    
            # Executa a consulta
            cursor.execute(sql)
    
            # Recupera os resultados
            resultados = cursor.fetchall()
            analysis_dir_path_arr = [row[0] for row in resultados]
            analysis_file_name_arr = [row[1] for row in resultados]
            analysis_db_path_arr = [row[2] for row in resultados]
    finally:
        # Fecha a conexão
        connection.close()
    return analysis_dir_path_arr, analysis_file_name_arr, analysis_db_path_arr

In [54]:
analysis_dir_path_arr, analysis_file_name_arr, analysis_db_path_arr = recuperar_dados_de_analise_do_bd_flag_2()

In [55]:
analysis_dir_path_arr

['D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L5T3_k7.8_MS25_SP107']

In [56]:
for analysis_dir_path, analysis_file_name, analysis_db_path in zip(analysis_dir_path_arr, analysis_file_name_arr, analysis_db_path_arr):
    executar_correcao(mapdl, analysis_dir_path, analysis_file_name, analysis_db_path, 1)

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Current working directory switched to D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L5T3_k0.525_MS25_SP113
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - CURRENT JOBNAME REDEFINED AS BL2M1P0.3L5T3_k7.8_MS25_SP107
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - RESUME ANSYS DATA FROM FILE NAME=BL2M1P0.3L5T3_k7.8_MS25_SP107.db                                                                                                                                                                                                                                    

 *** MAPDL GLOBAL STATUS *** 

 TITLE = BL2M1P0.3L5T3_k7.8_MS25_SP107                                         
 ANALYSIS TYPE =  STATIC (STEADY-STATE)
        66 KEYPOINTS DEFINED     MAX KEYPOINT NUMBER =             66
       127 LINES DEFINED         MAX LINE NUMBER =                147
        62 AREAS DEFINED         MAX AREA NUMBER =             

Widget(value='<iframe src="http://localhost:63992/index.html?ui=P_0x1c2b1ade330_72&reconnect=auto" class="pyvi…

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L5T3_k7.8_MS25_SP107
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_2_2_phi_0.3_lancanova_2024\BL2M1P0.3L5T3_k7.8_MS25_SP107


Widget(value='<iframe src="http://localhost:63992/index.html?ui=P_0x1c181d4a270_73&reconnect=auto" class="pyvi…

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = BL2M1P0.3L5T3_k7.8_MS25_SP107
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = rst
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_2_2_phi_0.3_lancanova_2024\BL2M1P0.3L5T3_k7.8_MS25_SP107


Widget(value='<iframe src="http://localhost:63992/index.html?ui=P_0x1c1e37b3380_74&reconnect=auto" class="pyvi…

n_u = 2451.3904
sigma_u = 250.1418775510204
w_max = 2.4732919909114104
von_mises_dist_img_path = D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L5T3_k7.8_MS25_SP107/BL2M1P0.3L5T3_k7.8_MS25_SP107_von_mises_dist.png
w_dist_img_path = D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L5T3_k7.8_MS25_SP107/BL2M1P0.3L5T3_k7.8_MS25_SP107_w_dist.png


Prosseguir com a persistência no BD? S


1 linhas foram atualizadas.
